# Analyse van AskDelphi Relations & Tags endpoint
Dit notebook ondersteunt het verkennen, analyseren en valideren van de Relations & Tags functionaliteit binnen AskDelphi.  
De module biedt helpermethoden voor het toevoegen, beheren en inspecteren van relaties en tags die aan AskDelphi‑topics gekoppeld zijn.  
Het doel van dit notebook is om:

- inzicht te krijgen in de datastructuren en API‑interacties rondom relations & tags
- voorbeelddata te analyseren en patronen te ontdekken
- helpermethoden te testen en gedrag te evalueren
- experimentele of onderzoeksgerichte analyses uit te voeren ter verbetering van de contentstructuur in AskDelphi

Dit notebook vormt daarmee een flexibel startpunt voor verdere exploratie of debugging van de Relations & Tags endpoint.

### Initialisatie omgeving

In [1]:
import sys
import uuid
import json
from pathlib import Path

project_root = Path.cwd().parent          # ../ vanaf Notebooks/
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

In [2]:
from pathlib import Path

# Vind de projectroot vanuit Notebooks/
project_root = Path.cwd().parent
src_path = project_root / "src"

# Voeg src/ toe aan modulezoekpad
if str(src_path) not in sys.path:
    sys.path.insert(0, str(src_path))

print("Project root:", project_root)
print("SRC path:", src_path)
print("SRC path is in sys.path?", str(src_path) in sys.path)

Project root: /Users/baasa03/projects/ask-delphi-api
SRC path: /Users/baasa03/projects/ask-delphi-api/src
SRC path is in sys.path? True


### Connectie met AskDelphi opzetten

In [3]:
from askdelphi.authentication import AskDelphiClient
client = AskDelphiClient()
client.authenticate()   # pakt automatisch portal code uit .env

/Users/baasa03/projects/ask-delphi-api/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


AskDelphi Client loaded.
Parsed tenant/project/acl from CMS URL
Loaded cached tokens.
AUTHENTICATION STARTED
Trying cached tokens...
Editing API token status: 200
Editing API token acquired.
SUCCESS using cached tokens!


True

In [4]:
from askdelphi.project import Project
from askdelphi.topictools import TopicTools
from askdelphi.relation import Relation
from askdelphi.workflow import Workflow
workflow = Workflow(client)
project = Project(client)
topic = TopicTools(client, project)
relation = Relation(client)

### Creatie Digicoach

In [5]:
# Constant
DIGICOACH_NAME = "Digicoach test arie 6feb"
TASK_NAME = "Taak test 6feb"
PREDEFINED_SEARCH_NAME = "Voorgedefinieerde zoekopdracht 6feb"

# Create Voorgedefinieerde zoekopdracht topic
topic_id_predefined_search = topic.topic_upload(PREDEFINED_SEARCH_NAME, "Pre-defined search")
topic_version_id_predefined_search = topic.get_topicVersionId(topic_id_predefined_search)
print(f"Created Voorgedefinieerde zoekopdracht topic : {topic_id_predefined_search}")

# Create Digicoach topic
topic_id_digicoach = str(uuid.uuid4())      
topicTitle = DIGICOACH_NAME      
topicTypeId = project.get_topic_type_id("Digitale Coach Procespagina")     
parentTopicId = topic_id_predefined_search
parentTopicRelationTypeId = relation.get_relation_type_id(topic_id_predefined_search, topic_version_id_predefined_search,"Voorgedefinieerde zoekopdracht")
parentTopicVersionId = topic_version_id_predefined_search
relation.add_topic_with_relation(client, topic_id_digicoach, topicTitle, topicTypeId, parentTopicId, parentTopicRelationTypeId, parentTopicVersionId)
print(f"Created Digicoach topic : {topic_id_digicoach}")

# Get Digicoach topic version ID
topic_version_id_digicoach = topic.get_topicVersionId(topic_id_digicoach)

# Create Task topic
topic_id_task = str(uuid.uuid4())
topicTitle = TASK_NAME       
topicTypeId = project.get_topic_type_id("Task")       
parentTopicId = topic_id_digicoach
parentTopicRelationTypeId = relation.get_relation_type_id(topic_id_digicoach, topic_version_id_digicoach, "Taak")
parentTopicVersionId = topic_version_id_digicoach
relation.add_topic_with_relation(client, topic_id_task, topicTitle, topicTypeId, parentTopicId, parentTopicRelationTypeId, parentTopicVersionId)
print(f"Created Task topic : {topic_id_task}")

# # Delete Task topic
# topic_version_id_task = topic.get_topicVersionId(topic_id_task)
# topic.delete_topic(topic_id_task, topic_version_id_task)
# print(f"Deleted Task topic : {topic_id_task}")

# # Delete Digicoach topic
# topic.delete_topic(topic_id_digicoach, topic_version_id_digicoach)
# print(f"Deleted Digicoach topic : {topic_id_digicoach}")

# # Delete Voorgedefinieerde zoekopdracht topic
# topic.delete_topic(topic_id_predefined_search, topic_version_id_predefined_search)
# print(f"Deleted Voorgedefinieerde zoekopdracht topic : {topic_id_predefined_search}")


REQUEST: GET https://edit.api.askdelphi.com/v1/tenant/0be6d42b-c278-44e6-888e-ba122840d690/project/397296f6-20dd-45cd-8459-250db2725140/acl/4ecd88f2-979b-4fb0-a95d-175d499bc375/contentdesign
RESPONSE: 200 OK

REQUEST: POST https://edit.api.askdelphi.com/v4/tenant/0be6d42b-c278-44e6-888e-ba122840d690/project/397296f6-20dd-45cd-8459-250db2725140/acl/4ecd88f2-979b-4fb0-a95d-175d499bc375/topic
JSON body: {
  "topicTitle": "Voorgedefinieerde zoekopdracht 6feb",
  "topicTypeId": "72aaefbb-1c09-4f89-8830-54194b03523b"
}
RESPONSE: 200 OK

REQUEST: POST https://edit.api.askdelphi.com/v1/tenant/0be6d42b-c278-44e6-888e-ba122840d690/project/397296f6-20dd-45cd-8459-250db2725140/acl/4ecd88f2-979b-4fb0-a95d-175d499bc375/topic/419e5f91-58bc-4f30-881c-b8cab4e30073/workflowstate
JSON body: {
  "action": 1
}
RESPONSE: 200 OK
Created Voorgedefinieerde zoekopdracht topic : 419e5f91-58bc-4f30-881c-b8cab4e30073

REQUEST: GET https://edit.api.askdelphi.com/v1/tenant/0be6d42b-c278-44e6-888e-ba122840d690/proje

### Tag functies uitzoeken

In [6]:
# import json
# # Type Digitale Coach Procespagina, Administratief beroep (Taskforce)
# topicId = "523c02c2-80e2-41e1-8188-10477e02f944"
# endpoint = f"/v1/tenant/{{tenantId}}/project/{{projectId}}/acl/{{aclEntryId}}/topic/{topicId}/tag"

# result = client._request("GET", endpoint)
# print(json.dumps(result, indent=2, ensure_ascii=False))

### Tag Digicoach

In [7]:
# Tag Digitale Coach Procespagina -> Keten Inning en betalingsverkeer

data = {
    "hierarchyTopicId": "81edcdae-337a-4c11-a275-8f2a85be1033",
    "hierarchyTopicTitle": "Ketens",
    "hierarchyNodeId": "88595ffc-4343-4343-9f13-8871665c6445",
    "hierarchyNodeTitle": "Inning en betalingsverkeer",
    "pathToNode": "",
    "hierarchyParentNodeId": "88595ffc-4343-4343-9f13-8871665c6445",
    "tagId": "00000000-0000-0000-0000-000000000000",
    "enforcedByAcl": False,
    "isTopicListFilter": False,
}

endpoint = f"/v2/tenant/{{tenantId}}/project/{{projectId}}/acl/{{aclEntryId}}/topic/{topic_id_digicoach}/topicVersion/{topic_version_id_digicoach}/tag"
client._request("POST", endpoint, json_data=data)

endpoint = f"/v2/tenant/{{tenantId}}/project/{{projectId}}/acl/{{aclEntryId}}/topic/{topic_id_predefined_search}/topicVersion/{topic_version_id_predefined_search}/tag"
client._request("POST", endpoint, json_data=data)


REQUEST: POST https://edit.api.askdelphi.com/v2/tenant/0be6d42b-c278-44e6-888e-ba122840d690/project/397296f6-20dd-45cd-8459-250db2725140/acl/4ecd88f2-979b-4fb0-a95d-175d499bc375/topic/f97cb855-0ae5-456f-b034-65ea6f1b2925/topicVersion/4fc445e5-57ce-43e3-b101-d391579b4cf0/tag
JSON body: {
  "hierarchyTopicId": "81edcdae-337a-4c11-a275-8f2a85be1033",
  "hierarchyTopicTitle": "Ketens",
  "hierarchyNodeId": "88595ffc-4343-4343-9f13-8871665c6445",
  "hierarchyNodeTitle": "Inning en betalingsverkeer",
  "pathToNode": "",
  "hierarchyParentNodeId": "88595ffc-4343-4343-9f13-8871665c6445",
  "tagId": "00000000-0000-0000-0000-000000000000",
  "enforcedByAcl": false,
  "isTopicListFilter": false
}
RESPONSE: 200 OK

REQUEST: POST https://edit.api.askdelphi.com/v2/tenant/0be6d42b-c278-44e6-888e-ba122840d690/project/397296f6-20dd-45cd-8459-250db2725140/acl/4ecd88f2-979b-4fb0-a95d-175d499bc375/topic/419e5f91-58bc-4f30-881c-b8cab4e30073/topicVersion/cdd0eae1-f5fe-49f5-95d9-8e44dd94d0c3/tag
JSON body: 

{'success': True,
 'errorCode': None,
 'errorMessage': None,
 'response': {'tags': []}}

In [8]:
# Tag Digitale Coach Procespagina -> Organisatieonderdeel MKB (Midden- en Kleinbedrijf)

data = {
    "hierarchyTopicId": "fe932c90-e383-48c6-a3a5-beaa41ebe3a3",
    "hierarchyTopicTitle": "Organisatieonderdelen",
    "hierarchyNodeId": "3e01fd9d-484a-4009-bbd4-95c9cf8df20a",
    "hierarchyNodeTitle": "MKB (Midden- en Kleinbedrijf)",
    "pathToNode": "",
    "hierarchyParentNodeId": "3e01fd9d-484a-4009-bbd4-95c9cf8df20a",
    "tagId": "00000000-0000-0000-0000-000000000000",
    "enforcedByAcl": False,
    "isTopicListFilter": False,
}

endpoint = f"/v2/tenant/{{tenantId}}/project/{{projectId}}/acl/{{aclEntryId}}/topic/{topic_id_digicoach}/topicVersion/{topic_version_id_digicoach}/tag"
client._request("POST", endpoint, json_data=data)

endpoint = f"/v2/tenant/{{tenantId}}/project/{{projectId}}/acl/{{aclEntryId}}/topic/{topic_id_predefined_search}/topicVersion/{topic_version_id_predefined_search}/tag"
client._request("POST", endpoint, json_data=data)


REQUEST: POST https://edit.api.askdelphi.com/v2/tenant/0be6d42b-c278-44e6-888e-ba122840d690/project/397296f6-20dd-45cd-8459-250db2725140/acl/4ecd88f2-979b-4fb0-a95d-175d499bc375/topic/f97cb855-0ae5-456f-b034-65ea6f1b2925/topicVersion/4fc445e5-57ce-43e3-b101-d391579b4cf0/tag
JSON body: {
  "hierarchyTopicId": "fe932c90-e383-48c6-a3a5-beaa41ebe3a3",
  "hierarchyTopicTitle": "Organisatieonderdelen",
  "hierarchyNodeId": "3e01fd9d-484a-4009-bbd4-95c9cf8df20a",
  "hierarchyNodeTitle": "MKB (Midden- en Kleinbedrijf)",
  "pathToNode": "",
  "hierarchyParentNodeId": "3e01fd9d-484a-4009-bbd4-95c9cf8df20a",
  "tagId": "00000000-0000-0000-0000-000000000000",
  "enforcedByAcl": false,
  "isTopicListFilter": false
}
RESPONSE: 200 OK

REQUEST: POST https://edit.api.askdelphi.com/v2/tenant/0be6d42b-c278-44e6-888e-ba122840d690/project/397296f6-20dd-45cd-8459-250db2725140/acl/4ecd88f2-979b-4fb0-a95d-175d499bc375/topic/419e5f91-58bc-4f30-881c-b8cab4e30073/topicVersion/cdd0eae1-f5fe-49f5-95d9-8e44dd94d0

{'success': True,
 'errorCode': None,
 'errorMessage': None,
 'response': {'tags': []}}

In [9]:
# Tag Digitale Coach Procespagina -> Documenttype Digitale coach

data = {
    "hierarchyTopicId": "f80a0410-8278-4f1c-86d4-efd51804d1ec",
    "hierarchyTopicTitle": "Documenttype",
    "hierarchyNodeId": "0560fee8-7b99-4b16-b2a6-3445e410bfee",
    "hierarchyNodeTitle": "Digitale coach",
    "pathToNode": "",
    "hierarchyParentNodeId": "0560fee8-7b99-4b16-b2a6-3445e410bfee",
    "tagId": "00000000-0000-0000-0000-000000000000",
    "enforcedByAcl": False,
    "isTopicListFilter": False,
}

endpoint = f"/v2/tenant/{{tenantId}}/project/{{projectId}}/acl/{{aclEntryId}}/topic/{topic_id_digicoach}/topicVersion/{topic_version_id_digicoach}/tag"
client._request("POST", endpoint, json_data=data)

endpoint = f"/v2/tenant/{{tenantId}}/project/{{projectId}}/acl/{{aclEntryId}}/topic/{topic_id_predefined_search}/topicVersion/{topic_version_id_predefined_search}/tag"
client._request("POST", endpoint, json_data=data)


REQUEST: POST https://edit.api.askdelphi.com/v2/tenant/0be6d42b-c278-44e6-888e-ba122840d690/project/397296f6-20dd-45cd-8459-250db2725140/acl/4ecd88f2-979b-4fb0-a95d-175d499bc375/topic/f97cb855-0ae5-456f-b034-65ea6f1b2925/topicVersion/4fc445e5-57ce-43e3-b101-d391579b4cf0/tag
JSON body: {
  "hierarchyTopicId": "f80a0410-8278-4f1c-86d4-efd51804d1ec",
  "hierarchyTopicTitle": "Documenttype",
  "hierarchyNodeId": "0560fee8-7b99-4b16-b2a6-3445e410bfee",
  "hierarchyNodeTitle": "Digitale coach",
  "pathToNode": "",
  "hierarchyParentNodeId": "0560fee8-7b99-4b16-b2a6-3445e410bfee",
  "tagId": "00000000-0000-0000-0000-000000000000",
  "enforcedByAcl": false,
  "isTopicListFilter": false
}
RESPONSE: 200 OK

REQUEST: POST https://edit.api.askdelphi.com/v2/tenant/0be6d42b-c278-44e6-888e-ba122840d690/project/397296f6-20dd-45cd-8459-250db2725140/acl/4ecd88f2-979b-4fb0-a95d-175d499bc375/topic/419e5f91-58bc-4f30-881c-b8cab4e30073/topicVersion/cdd0eae1-f5fe-49f5-95d9-8e44dd94d0c3/tag
JSON body: {
  "h

{'success': True,
 'errorCode': None,
 'errorMessage': None,
 'response': {'tags': []}}

In [10]:
# Checkin Voorgedefinieerde zoekopdracht topic
topic.checkin(topic_id_predefined_search)
print(f"Checked in Voorgedefinieerde zoekopdracht topic : {topic_id_predefined_search}")

# Checkin Digicoach topic
topic.checkin(topic_id_digicoach)
print(f"Checked in Digicoach topic : {topic_id_digicoach}")


REQUEST: POST https://edit.api.askdelphi.com/v1/tenant/0be6d42b-c278-44e6-888e-ba122840d690/project/397296f6-20dd-45cd-8459-250db2725140/acl/4ecd88f2-979b-4fb0-a95d-175d499bc375/topic/419e5f91-58bc-4f30-881c-b8cab4e30073/workflowstate
JSON body: {
  "action": 0
}
RESPONSE: 200 OK
Checked in Voorgedefinieerde zoekopdracht topic : 419e5f91-58bc-4f30-881c-b8cab4e30073

REQUEST: POST https://edit.api.askdelphi.com/v1/tenant/0be6d42b-c278-44e6-888e-ba122840d690/project/397296f6-20dd-45cd-8459-250db2725140/acl/4ecd88f2-979b-4fb0-a95d-175d499bc375/topic/f97cb855-0ae5-456f-b034-65ea6f1b2925/workflowstate
JSON body: {
  "action": 0
}
RESPONSE: 200 OK
Checked in Digicoach topic : f97cb855-0ae5-456f-b034-65ea6f1b2925


### Publiceer Digicoach

In [11]:
#  Creates a workflow transition request for predefined_search topic.
request_id = workflow.create_workflow_transition_request(topic_id_predefined_search)
transitions_model = workflow.get_workflow_transition_request_transitions_model(request_id)
workflow.update_workflow_transition_request(request_id, transitions_model)
workflow.approve_workflow_transition_request(request_id)

#  Creates a workflow transition request for digicoach topic.
request_id = workflow.create_workflow_transition_request(topic_id_digicoach)
transitions_model = workflow.get_workflow_transition_request_transitions_model(request_id)
workflow.update_workflow_transition_request(request_id, transitions_model)
workflow.approve_workflow_transition_request(request_id)

#  Creates a workflow transition request for task topic.
request_id = workflow.create_workflow_transition_request(topic_id_task)
transitions_model = workflow.get_workflow_transition_request_transitions_model(request_id)
workflow.update_workflow_transition_request(request_id, transitions_model)
workflow.approve_workflow_transition_request(request_id)


REQUEST: POST https://edit.api.askdelphi.com/v1/tenant/0be6d42b-c278-44e6-888e-ba122840d690/project/397296f6-20dd-45cd-8459-250db2725140/acl/4ecd88f2-979b-4fb0-a95d-175d499bc375/workflow-transition-request/topic/419e5f91-58bc-4f30-881c-b8cab4e30073
JSON body: {
  "Url": "https://digitalecoach.askdelphi.com/cms/tenant/0be6d42b-c278-44e6-888e-ba122840d690/project/397296f6-20dd-45cd-8459-250db2725140/acl/4ecd88f2-979b-4fb0-a95d-175d499bc375"
}
RESPONSE: 200 OK

REQUEST: GET https://edit.api.askdelphi.com/v1/tenant/0be6d42b-c278-44e6-888e-ba122840d690/project/397296f6-20dd-45cd-8459-250db2725140/acl/4ecd88f2-979b-4fb0-a95d-175d499bc375/workflow-transition-request/359ee16a-42c5-4b4f-a759-ece03192956b/transitions
RESPONSE: 200 OK

REQUEST: PUT https://edit.api.askdelphi.com/v1/tenant/0be6d42b-c278-44e6-888e-ba122840d690/project/397296f6-20dd-45cd-8459-250db2725140/acl/4ecd88f2-979b-4fb0-a95d-175d499bc375/workflow-transition-request/359ee16a-42c5-4b4f-a759-ece03192956b/transitions
JSON body:

{}